(cat-data-notebook)=
# Dati non gaussiani e categoriali

In [ ]:
source("_common.R")
suppressPackageStartupMessages({
    library("lavaan")
    library("semPlot")
    library("knitr")
    library("markdown")
    library("patchwork")
    library("here")
})

set.seed(42)

Negli esempi precedenti di questa dispensa è stato utilizzato lo stimatore di massima verosimiglianza (ML). Molti dei modelli CFA e SEM riportati nella letteratura di ricerca applicata utilizzano infatti stime di ML. Tuttavia, lo stimatore ML è appropriata solo per dati multivariati normali a livello di scala a intervalli (cioè, quando la distribuzione congiunta delle variabili continue è distribuita normalmente). Quando i dati continui si discostano marcatamente dalla normalità (cioè, mostrano una forte asimmetria o curtosi), o quando alcuni degli indicatori non sono a livello di scala a intervalli (cioè, dati binari, politomici o ordinali), allora è preferibile usare uno stimatore diverso da quello di ML.

## Dati non gaussiani

La ricerca ha dimostrato che la stima di ML è robusta nel caso di piccole deviazioni nella normalità. Tuttavia, quando la non normalità è più pronunciata, è necessario utilizzare uno stimatore diverso dalla ML per ottenere risultati statistici affidabili (vale a dire, statistiche accurate sulla bontà dell'adattamento ed errori standard accurati delle stime dei parametri). La stima di ML è particolarmente sensibile ad un eccesso di curtosi.

Le conseguenze dell'uso della ML in condizioni di grave non normalità includono 

- valori eccessivi della statistica $\chi^2$ del modello; 
- la sottostima degli indici di bontà dell'adattamento mediante indici quali TLI e CFI;
- la sottostima degli errori standard dei parametri.

Questi effetti deleteri sono esacerbati dalla diminuzione della dimensione del campione. 

Allo scopo di limitare tali conseguenze indesiderate, nelle condizioni di marcata violazione dell'assunzione di normalità multivariata, vengono usati stimatori diversi dalla ML. I due stimatori più comunemente usati per dati continui non normali sono 

- ML robusto,
- minimi quadrati ponderati (WLS). 

L'uso di WLS non è, in generale, raccomandato, a meno che le dimensioni del campione non siano molto grandi. Al contrario, la ricerca ha dimostrato che il metodo ML robusto fornisce uno stimatore adeguato rispetto a diversi livelli di non normalità. 

Esaminiamo qui un esempio discusso da {cite:t}`brown2015confirmatory` nelle sue tabelle 9.5 -- 9.7. 

In [ ]:
d <- readRDS(here::here("data", "brown_table_9_5_data.RDS"))
head(d)

Le statistiche descrittive di questo campione di dati mostrano valori eccessivi di asimmetria e di curtosi.

In [ ]:
psych::describe(d)

Definiamo un modello ad un fattore e, seguendo @brown2015confirmatory, aggiungiamo una correlazione residua tra gli indicatori `X1` e `X3`:

In [ ]:
model <- '
  f1 =~ x1 + x2 + x3 + x4 + x5
  x1 ~~ x3 
'

Procediamo alla stima dei parametri utilizzando uno stimatore di ML robusto. La sintassi `lavaan` è la seguente:

In [ ]:
fit <- cfa(model, data = d, mimic = "MPLUS", estimator = "MLM")

Per esaminare la soluzione ottenuta ci focalizziamo sulla statistica $\chi^2$ -- si consideri la soluzione robusta fornita nell'output.

In [ ]:
out = summary(fit)
print(out)

Per fare un confronto, adattiamo lo stesso modello ai dati usando lo stimatore di ML.

In [ ]:
fit2 <- cfa(model, data = d)

Notiamo come il valore della statistica $\chi^2$ ora ottenuto sia molto maggiore di quello trovato in precedenza.

In [ ]:
out = summary(fit2)
print(out)

## Dati categoriali

Quando almeno un indicatore è categoriale (cioè binario, politomico o ordinale), il metodo ML ordinario non dovrebbe essere utilizzato per stimare i modelli CFA. Vi sono molte potenziali conseguenze del trattamento delle variabili categoriali come continue in un'analisi CFA, incluso il fatto che può tale scelta può 

- produrre stime attenuate delle relazioni tra gli indicatori, specialmente quando ci sono effetti pavimento o soffitto;
- portare ad individuare "pseudo-fattori" che emergono come artefatti del metodo statistico;  
- produrre distorsioni negli indici di bontà dell'adattamento e nelle stime degli errori standard;
- produrre stime errate dei parametri.

Esistono vari stimatori che possono essere utilizzati con indicatori categoriali; ad esempio, gli stimatori dei minimi quadrati ponderati (WLS), dei minimi quadrati ponderati robusti (WLSMV) e dei minimi quadrati non ponderati (ULS).

### Un esempio concreto

Nell'esempio discusso da @brown2015confirmatory, i ricercatori desiderano verificare un modello uni-fattoriale di dipendenza da alcol in un campione di 750 pazienti ambulatoriali. Gli indicatori di alcolismo sono item binari che riflettono la presenza/assenza di sei criteri diagnostici per l'alcolismo (0 = criterio non soddisfatto, 1 = criterio soddisfatto). I dati sono i seguenti:

In [ ]:
d1 <- readRDS(here::here("data", "brown_table_9_9_data.RDS"))
head(d1)

Il modello viene specificato nel modo seguente:

In [ ]:
model1 <- '
  etoh =~ y1 + y2 + y3 + y4 + y5 + y6
'

Adattiamo il modello specificando che i dati sono a livello di scala ordinale (stimatore WLSMVS).

In [ ]:
fit1 <- cfa(
  model1, 
  data = d1, 
  ordered = names(d1), 
  estimator = "WLSMVS", 
  mimic = "mplus"
)

Esaminiamo la soluzione ottenuta:

In [ ]:
out = summary(fit1, fit.measures = TRUE)
print(out)

Confrontiamo la soluzione ottenuta con lo stimatore WLSMVS con quella ottenuta mediante lo stimatore ML.

In [ ]:
fit2 <- cfa(
  model1, 
  data = d1
)

In [ ]:
out = summary(fit2, fit.measures = TRUE)
print(out)

Si noti che la soluzione ottenuta mediante lo stimatore WLSMVS produce indici di bontà di adattamento migliori e errori standard dei parametri più piccoli.
